In [ ]:
import os

In [ ]:
 ! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [ ]:
! mkdir dataset
! unzip chest-xray-pneumonia.zip -d dataset

In [ ]:
! mkdir ./dataset/chest_xray/chest_xray_normal
! mkdir ./dataset/chest_xray/chest_xray_pneumonia
! cp -r ./dataset/chest_xray/chest_xray/train/NORMAL ./dataset/chest_xray/chest_xray_normal/train
! cp -r ./dataset/chest_xray/chest_xray/train/PNEUMONIA ./dataset/chest_xray/chest_xray_pneumonia/train

In [ ]:
os.makedirs('generated/normal', exist_ok=True)
os.makedirs('generated/pneumonia', exist_ok=True)

In [ ]:
! pip install pillow

In [ ]:
from PIL import Image
import glob

In [ ]:
import random

In [ ]:
def crop_center(image, crop_width, crop_height, x=0,y=0):
    width, height = image.size
    return image.crop(((width + x - crop_width) // 2,
                         (height + y - crop_height) // 2,
                         (width + x + crop_width) // 2,
                         (height + y + crop_height) // 2))

In [ ]:
def resize(image, size=128):
  width, height = image.size
  min_dimension = height if width > height else width
  image = crop_center(image, min_dimension, min_dimension)
  image = image.resize((size, size))
  return image

def random_zoom(image):
  rotation = random.normalvariate(0, 3)
  image = image.rotate(rotation)
  image = resize(image, random.randint(140,168))
  y_translate = random.normalvariate(0, 3)
  x_translate = random.normalvariate(0, 3)
  width, height = image.size
  min_dimension = height if width > height else width
  image = crop_center(image, min_dimension, min_dimension, y_translate, x_translate)
  image = image.resize((128, 128))
  return image

In [ ]:
def random_transform(image):
  zoomed = bool(random.getrandbits(1))
  image = random_zoom(image) if zoomed else resize(image)

  return image


In [ ]:
images = glob.glob("dataset/chest_xray/chest_xray_normal/train/*.jpeg")
for idx, image in enumerate(images):
    with open(image, 'rb') as file:
        img = Image.open(file)
        for x in range(5):
          transformed_img = random_transform(img)
          transformed_img.save('generated/normal/transformed-xray-{0:0=5d}-{1}.jpg'.format(idx,x))

images = glob.glob("dataset/chest_xray/chest_xray_pneumonia/train/*.jpeg")
for idx, image in enumerate(images):
    with open(image, 'rb') as file:
        img = Image.open(file)
        for x in range(5):
          transformed_img = random_transform(img)
          transformed_img.save('generated/pneumonia/transformed-xray-{0:0=5d}-{1}.jpg'.format(idx,x))
